1. 정상/비정상 데이터 정의
- 정상 데이터: Lying 상태 데이터.
- 비정상 데이터: 특정 활동 (예: Walking 또는 다른 Activity) 데이터.

2. 오토인코더
- 대칭형 (Encoder와 Decoder 구조 동일).
- 비대칭형 (Encoder와 Decoder 구조 다름).

3. DAE(Denoising Autoencoder)
- 노이즈를 추가한 입력으로 학습하여 모델의 이상치 탐지 성능 확인.

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
import numpy as np

In [18]:
dsa_data = pd.read_csv("C:/datasets/DSA_features.csv")

정상 및 비정상 데이터 분류

정상 데이터로 간주할 활동: "lyingBack", "lyingRigh".
비정상 데이터로 간주할 활동: "runningTreadmill".

In [19]:
normal_activities = ["lyingBack", "lyingRigh"]
abnormal_activity = "runningTreadmill"

In [20]:
normal_data = dsa_data[dsa_data["activity"].isin(normal_activities)]
abnormal_data = dsa_data[dsa_data["activity"] == abnormal_activity]

In [21]:
#데이터셋의 마지막 두 열(activity와 people)을 제외한 모든 열을 피처로 설정.
feature_columns = dsa_data.columns[:-2]

In [22]:
X_normal = normal_data[feature_columns]
X_abnormal = abnormal_data[feature_columns]

X_normal: 정상 데이터의 피처 값만 추출.
X_abnormal: 비정상 데이터의 피처 값만 추출.

In [24]:
scaler = MinMaxScaler()
X_normal_scaled = scaler.fit_transform(X_normal)
X_abnormal_scaled = scaler.transform(X_abnormal)

MinMaxScaler로 모든 피처 값을 0~1 범위로 정규화.

fit_transform: 정상 데이터에 대해 스케일링 기준을 학습(fit)하고 데이터를 변환(transform).

transform: 학습한 기준으로 비정상 데이터를 변환.

In [25]:
X_train, X_test = train_test_split(X_normal_scaled, test_size=0.2, random_state=42)

대칭형 오토인코더의 정의:

1. 입력 차원(input_dim)을 받음.

2. 128 → 64 → 128 → input_dim의 대칭 구조.

3. 각 층은 활성화 함수(relu 또는 sigmoid)를 사용.

4. 손실 함수로 MSE(Mean Squared Error)를 사용하여 입력 데이터를 재구성.

In [26]:
def build_symmetric_autoencoder(input_dim):
    """Builds a symmetric autoencoder."""
    autoencoder = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dense(64, activation='relu'),
        Dense(128, activation='relu'),
        Dense(input_dim, activation='sigmoid')
    ])
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

비대칭형 오토인코더의 정의:

1. 입력 차원(input_dim)을 받음.
2. 128 → 32 → 64 → input_dim의 비대칭 구조.

In [27]:
def build_asymmetric_autoencoder(input_dim):
    """Builds an asymmetric autoencoder."""
    autoencoder = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dense(32, activation='relu'),
        Dense(64, activation='relu'),
        Dense(input_dim, activation='sigmoid')
    ])
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

DAE (Denoising Autoencoder) 정의:

노이즈를 추가하여 입력 데이터에서 일부 정보를 제거(Dropout)한 후 입력을 복원하도록 학습.

In [28]:
def build_dae(input_dim, noise_factor=0.1):
    """Builds a denoising autoencoder."""
    input_layer = Input(shape=(input_dim,))
    noisy_input = Dropout(noise_factor)(input_layer)
    encoded = Dense(128, activation='relu')(noisy_input)
    encoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(encoded)
    output_layer = Dense(input_dim, activation='sigmoid')(decoded)
    dae = Model(inputs=input_layer, outputs=output_layer)
    dae.compile(optimizer='adam', loss='mse')
    return dae

In [30]:
input_dim = X_train.shape[1]

In [31]:
symmetric_ae = build_symmetric_autoencoder(input_dim)
symmetric_ae.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test), verbose=0)

In [32]:
asymmetric_ae = build_asymmetric_autoencoder(input_dim)
asymmetric_ae.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test), verbose=0)

In [33]:
dae = build_dae(input_dim, noise_factor=0.1)
dae.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test), verbose=0)

In [34]:
reconstructed_symmetric = symmetric_ae.predict(X_abnormal_scaled)
reconstructed_asymmetric = asymmetric_ae.predict(X_abnormal_scaled)
reconstructed_dae = dae.predict(X_abnormal_scaled)

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


비정상 데이터 이상치 탐지 : 비정상 데이터를 각각의 오토인코더에 입력하여 복원

In [35]:
loss_symmetric = np.mean(np.mean(np.square(X_abnormal_scaled - reconstructed_symmetric), axis=1))
loss_asymmetric = np.mean(np.mean(np.square(X_abnormal_scaled - reconstructed_asymmetric), axis=1))
loss_dae = np.mean(np.mean(np.square(X_abnormal_scaled - reconstructed_dae), axis=1))

비정상 데이터의 MSE 계산: 복원된 출력과 실제 입력 간의 차이를 평균제곱오차로 계산

In [36]:
loss_symmetric, loss_asymmetric, loss_dae

(7626.879644458769, 7608.2895439847425, 7607.435556447733)